In [3]:
# cd to the root folder
# CHANGE
%cd ~/code/saoriyosano/poverty_mapper

/home/saori/code/saoriyosano/poverty_mapper


In [44]:
%load_ext autoreload
%autoreload 2 

In [46]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from glob import glob
from batchers import dataset_constants, tfrecord_paths_utils
from model import processing
from model.loss import r2_metric
import datetime

### Constants

In [104]:
DHS_TFRECORDS_PATH_ROOT = 'data/dhs_tfrecords/' # CHANGE
CSV_PATH = 'data/dhs_clusters.csv' # CHANGE
CLUSTERS_DF = pd.read_csv(CSV_PATH, float_precision='high', index_col=False)
IR_BANDS = ['NIR', 'SWIR1', 'SWIR2']
RGB_BANDS = ['RED', 'GREEN', 'BLUE']
MEANS = dataset_constants._MEANS_DHS
STDS = dataset_constants._STD_DEVS_DHS
BATCH_SIZE = 64
DATASET = [('angola_2011'), ()]
SHUFFLE = 50
PREFETCH = 2
EPOCHS = 5 # CHANGE
CHECKPOINT_PATH = 'model/checkpoints' # CHANGE

In [106]:
def process_datasets(dataset, shuffle, batch_size, prefetch):
    
    # train_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="train")
    # val_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="val")
    # test_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="test")

    # for testing - comment out
    tfrecord_paths = []
    for country, year in dataset: # SY: added this for testing
        country_year = f"{country}_{year}"
        glob_path = os.path.join(
            DHS_TFRECORDS_PATH_ROOT, country_year + '*', '*.tfrecord.gz')
        tfrecord_paths += glob(glob_path)
    tfrecord_paths = np.sort(tfrecord_paths)

    # for testing - comment out
    train_tfrecord_paths = tfrecord_paths[:3]
    val_tfrecord_paths = tfrecord_paths[3:6]
    test_tfrecord_paths = tfrecord_paths[:3]
    print(train_tfrecord_paths, val_tfrecord_paths, test_tfrecord_paths)

    train_ds = tf.data.TFRecordDataset(train_tfrecord_paths, compression_type="GZIP")
    val_ds = tf.data.TFRecordDataset(val_tfrecord_paths, compression_type="GZIP")
    test_ds = tf.data.TFRecordDataset(test_tfrecord_paths, compression_type="GZIP")

    # normalize and resize
    train_ds = train_ds.map(processing.process_tfrecords)
    val_ds = val_ds.map(processing.process_tfrecords)  
    test_ds = test_ds.map(processing.process_tfrecords)

    # train_ds = train_ds.map(processing.augment)
    # val_ds = val_ds.map(processing.augment)  
    # test_ds = test_ds.map(processing.augment)

    train_ds = train_ds.cache()
    train_ds = train_ds.shuffle(shuffle)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.prefetch(prefetch)

    val_ds = val_ds.cache()
    val_ds = val_ds.shuffle(shuffle)
    val_ds = val_ds.batch(batch_size)
    val_ds = val_ds.prefetch(prefetch)

    test_ds = test_ds.cache()
    test_ds = test_ds.shuffle(shuffle)
    test_ds = test_ds.batch(batch_size)
    test_ds = test_ds.prefetch(prefetch)

    return train_ds, val_ds, test_ds

In [107]:
def vgg16_model_train_val(train_ds, val_ds, epochs, checkpoint_path):
    checkpoint_path = checkpoint_path

    input_tensor = tf.keras.Input(shape=(224, 224, 3), name = 'images')
    vgg16_model = tf.keras.applications.vgg16.VGG16(include_top = False, input_tensor = input_tensor)    
    x = vgg16_model.output
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    predictions = tf.keras.layers.Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs=vgg16_model.input, outputs=predictions)
    model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])
    
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor = 'val_loss',
        verbose = 0,
        save_best_only = True,
        save_freq = 'epoch',
    )
    hist = model.fit(
        train_ds, epochs=epochs, validation_data=val_ds,
        callbacks=[cp_callback])
    
    # save the model
    trained_model = hist.model
    now = datetime.datetime.now()
    date_time = now.strftime("%Y%m%d-%H%M%S")
    model_name = f"{checkpoint_path}/model-{date_time}.h5"
    trained_model.save(model_name)

    return trained_model, hist.history, hist.params

In [108]:
# evaluate function to be implemented TODO
# def vgg16_test(test_ds, checkpoint_path, batch_size):
#     trained_model = model.load_weights(checkpoint_path)
#     trained_model.evaluate()

In [109]:
train_ds, val_ds, test_ds = process_datasets(DATASET, SHUFFLE, BATCH_SIZE, PREFETCH)

ValueError: too many values to unpack (expected 2)

In [ ]:
train_ds

<DatasetV1Adapter shapes: {images: (?, 224, 224, 3), y: (?,)}, types: {images: tf.float32, y: tf.float32}>

In [ ]:
trained_model, perf, params = vgg16_model_train_val(train_ds, val_ds, EPOCHS, CHECKPOINT_PATH)

Train on None steps
Epoch 1/5


ValueError: Empty training data.

In [75]:
def process_and_train(dataset, shuffle, batch_size, prefetch, bands, epochs, checkpoint_path):
    img_bands = bands
    train_ds, val_ds, test_ds = process_datasets(dataset, shuffle, batch_size, prefetch)
    trained_model, perf, params = vgg16_model_train_val(train_ds, val_ds, epochs, checkpoint_path)
    return trained_model, perf, params

### VGG16 Model: Infrared

In [76]:
trained, perf, params = process_and_train(
    dataset=DATASET, shuffle=SHUFFLE, batch_size=BATCH_SIZE, 
    prefetch=PREFETCH, bands=IR_BANDS, 
    epochs=EPOCHS, checkpoint_path=CHECKPOINT_PATH
    )

TypeError: Expected sequence or array-like, got <class 'tensorflow.python.framework.ops.Tensor'>

In [12]:
input_tensor = tf.keras.Input(shape=(224, 224, 3), name = 'images')
vgg16_model = tf.keras.applications.vgg16.VGG16(include_top = False, input_tensor = input_tensor)
x = vgg16_model.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='linear')(x)
model = tf.keras.Model(inputs=vgg16_model.input, outputs=predictions)
model.compile(loss='mse', optimizer='adam', metrics=[r2_metric])
model.fit(train_ds, epochs=2, validation_data=)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2023-03-03 17:12:08.460400: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-03 17:12:08.460490: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-03 17:12:08.460517: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Chau): /proc/driver/nvidia/version does not exist
2023-03-03 17:12:08.465928: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-03-03 17:12:08.475811: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2711995000 Hz
2023-03-03 17:12:08.476931: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cbde8c60e0 initialized for platform Host (this does not guarantee that XLA wi

ValueError: Unknown metric function:r2